In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from sbi.inference import prepare_for_sbi, SNPE
from sbi.simulators.linear_gaussian import diagonal_linear_gaussian
import sbi.utils as sbi_utils

pyknos/nflows/nn/nde


In [3]:
import pickle

In [9]:
with open('file', 'rb') as f:
    x = pickle.load(f, encoding='latin1')

FileNotFoundError: [Errno 2] No such file or directory: 'file'

### Load prior min and max and gt

In [3]:
import sys
sys.path.append('../')

In [4]:
from parameter_setup import load_ground_truth_params, load_prior_min, load_prior_max

### Create dummy data

In [5]:
num_samples = 10000
x = torch.randn(num_samples, 13)
theta = torch.randn(num_samples, 35)

### Run simulator to get x_o

In [6]:
x_o = torch.zeros(1, 13)

### Create dummy simulator and dummy prior

In [7]:
def dummy_simulator(theta):
    return torch.ones(1,13)

dummy_prior = sbi_utils.BoxUniform(torch.as_tensor(load_prior_min()), torch.as_tensor(load_prior_max()))
_bound = torch.sqrt(torch.as_tensor(3.))
dummy_prior_norm = sbi_utils.BoxUniform(-_bound*torch.ones(35), _bound*torch.ones(35))

In [17]:
simulator, prior, x_shape = prepare_for_sbi(dummy_simulator, dummy_prior_norm)

In [25]:
from bflows.utils.neural_net.get_bounded_flows import get_bflow

boxFlow = get_bflow("nsf_bounded", prior=dummy_prior_norm, context=x_o)

In [26]:
inference = SNPE(
    simulator,
    prior, 
    x_shape,
    density_estimator=boxFlow,
    external_data=(theta, x),
)

In [27]:
posterior = inference(
    num_rounds=1,
    num_simulations_per_round=0,
    max_num_epochs=5,
    z_score_x=False
)

hereee=============  [tensor([[True],
        [True],
        [True],
        ...,
        [True],
        [True],
        [True]])]


### Analyse posterior

In [12]:
posterior = posterior.set_default_x(x=x_o)

In [13]:
samples = posterior.sample((10000,), show_progress_bars=True)

# convert to list for pickling
samples_list = samples.numpy().tolist()

In [29]:
import pickle
with open('../results/posteriors/200616_dummyPosteriorSNL.pickle', 'wb') as handle:
    pickle.dump(posterior, handle)

In [36]:
with open('../results/samples/200616_dummyPosteriorSNL_samples.pickle', 'wb') as handle:
    pickle.dump(samples_list, handle, protocol=2)

### Posterior predictives
Has to be done in a different virtual env with python 2.